In [ ]:
import torch
import torchvision
from torch.autograd import Variable as V
from visualize import make_dot

In [ ]:
resnet_18 = torchvision.models.resnet18(pretrained=True)
resnet_18.eval();

In [ ]:
x = V(torch.randn(1, 3, 224, 224))#, volatile=True)
h_x = resnet_18.forward(x)

In [ ]:
dot = make_dot(h_x)
dot.render('net.dot')
# dot  # uncomment for displaying the graph

In [ ]:
print(h_x.creator)
print(type(h_x.creator.previous_functions))
print(len(h_x.creator.previous_functions))
print('---')
for a, b in enumerate(h_x.creator.previous_functions): print(a, '-->', b)
print('---')
last_view = h_x.creator.previous_functions[0][0]

In [ ]:
handle = last_view.register_forward_hook(lambda: print('running forward'))

In [ ]:
print(resnet_18)

In [ ]:
last_pool = last_view.previous_functions[0][0]
type(last_pool)

In [ ]:
resnet_18._modules.keys()

In [ ]:
avgpool_layer = resnet_18._modules.get('avgpool')

In [ ]:
h = avgpool_layer.register_forward_hook(
        lambda m,i,o: \
        print(
            '\nm:', type(m),
            '\ni:', type(i),
                '\n   len:', len(i),
                '\n   type:', type(i[0]),
                '\n   data size:', i[0].data.size(),
                '\n   data type:', i[0].data.type(),
            '\no:', type(o),
                '\n   data size:', o.data.size(),
                '\n   data type:', o.data.type(),
        )
)

In [ ]:
h_x = resnet_18(x)

In [ ]:
h.remove()

In [ ]:
my_embedding = torch.zeros(512)
h = avgpool_layer.register_forward_hook(
        lambda m,i,o: my_embedding.copy_(o.data)
)
h_x = resnet_18(x)
h.remove()

In [ ]:
avgpool_layer.